In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

if os.environ["OPENAI_API_KEY"]:
    print('API Key is set')

API Key is set


In [2]:
from langchain_openai import ChatOpenAI 

In [3]:
llm=ChatOpenAI(model="gpt-5-nano", temperature=0)

In [4]:
# Test the LLM
response=llm.invoke('what is RAG in one line?')
response.content

'RAG (Retrieval-Augmented Generation) is a model approach that retrieves relevant documents from a datastore and conditions its text generation on those retrieved documents.'

In [5]:
from langchain_core.documents import Document

## Text data
my_text="""
Company Profile: test_123
About Us Established in the bustling industrial hub of Faridabad, Haryana, test_123 is a premier shopping mart dedicated to redefining the local retail experience. We offer a comprehensive selection of high-quality groceries, daily essentials, and lifestyle products, bridging the gap between traditional markets and modern convenience. Our store is deeply rooted in the community, prioritizing partnerships with local farmers to source fresh, organic produce while maintaining competitive pricing for families. With a customer-first philosophy, test_123 has grown into a trusted household name, committed to sustainability and exceptional service across the region.
Financial Performance Summary (Fiscal Year 2025)
During the first quarter of 2025, test_123 started with a strong footing, generating a revenue of ₹45,00,000 and securing a profit of ₹5,20,000. The second quarter presented some challenges, resulting in a revenue of ₹38,50,000 and a net loss of (₹1,50,000). However, the company recovered significantly in the third quarter, posting a revenue of ₹52,00,000 with a healthy profit of ₹8,10,000. The fiscal year concluded on a high note in the fourth quarter, achieving a peak revenue of ₹61,00,000 and a profit of ₹12,40,000.
"""

docs=[Document(page_content=my_text,metadata={"source":"test_123_company","document_id":"101"})]
docs

[Document(metadata={'source': 'test_123_company', 'document_id': '101'}, page_content='\nCompany Profile: test_123\nAbout Us Established in the bustling industrial hub of Faridabad, Haryana, test_123 is a premier shopping mart dedicated to redefining the local retail experience. We offer a comprehensive selection of high-quality groceries, daily essentials, and lifestyle products, bridging the gap between traditional markets and modern convenience. Our store is deeply rooted in the community, prioritizing partnerships with local farmers to source fresh, organic produce while maintaining competitive pricing for families. With a customer-first philosophy, test_123 has grown into a trusted household name, committed to sustainability and exceptional service across the region.\nFinancial Performance Summary (Fiscal Year 2025)\nDuring the first quarter of 2025, test_123 started with a strong footing, generating a revenue of ₹45,00,000 and securing a profit of ₹5,20,000. The second quarter pr

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter= RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50)

chunks=splitter.split_documents(docs)
chunks

[Document(metadata={'source': 'test_123_company', 'document_id': '101'}, page_content='Company Profile: test_123'),
 Document(metadata={'source': 'test_123_company', 'document_id': '101'}, page_content='About Us Established in the bustling industrial hub of Faridabad, Haryana, test_123 is a premier shopping mart dedicated to redefining the local retail experience. We offer a comprehensive selection of high-quality groceries, daily essentials, and lifestyle products, bridging the gap between traditional markets and modern convenience. Our store is deeply rooted in the community, prioritizing partnerships with local farmers to source fresh, organic produce while maintaining competitive pricing'),
 Document(metadata={'source': 'test_123_company', 'document_id': '101'}, page_content='produce while maintaining competitive pricing for families. With a customer-first philosophy, test_123 has grown into a trusted household name, committed to sustainability and exceptional service across the re

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings_model=OpenAIEmbeddings(model="text-embedding-3-small")

In [8]:
from langchain_community.vectorstores import Chroma

vectorstore=Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    persist_directory='./CompanyDB/'
    )

In [9]:
context=vectorstore.similarity_search('What test_123 company do?',k=3)

In [10]:
response=llm.invoke(f'what is Q1 revenue of test_123 company?: {context}')
print(response.content)

Q1 revenue: ₹45,00,000 (₹4.5 million).


In [13]:
response=llm.invoke(f'what is Q1 profite of test_123 company?: {context}')
print(response.content)

Q1 profit (Q1 2025) = ₹5,20,000 (₹520,000).


In [14]:
response=llm.invoke(f'test_123 company next year projection?: {context}')
print(response.content)

Here’s a rough, data‑driven projection for test_123 for the next year, based on the 2025 quarterly data you provided. Note this is a simple forecast using observed revenue behavior and margin patterns; actual results can vary with seasonality, costs, and market factors. Also, Q4 2025 profit figure isn’t given, so margins are derived from Q1–Q3.

Key 2025 figures (for reference)
- Total 2025 revenue (Q1–Q4): ₹19.65 lakh? Wait — in crores: ₹1.965 crore (₹19.65 million).
- Quarterly revenues: Q1 ₹4.50 million, Q2 ₹3.85 million, Q3 ₹5.20 million, Q4 ₹6.10 million.
- Quarterly margins (profit/revenue): Q1 ~11.6%, Q2 ~-3.9%, Q3 ~15.6%. Average of Q1–Q3 margin ≈ 7.75%.

Projections for 2026 (rough, three scenarios)

1) Base-case (most balanced)
- Assumptions: roughly 12–13% annual revenue growth; margin similar to the Q1–Q3 average (~7.75%).
- 2026 revenue: approximately ₹2.21 crore (₹2.21–2.22 crore).
- 2026 profit: about 7.75% of revenue ≈ ₹17–18 lakh.
- Summary: Revenue ≈ ₹2.21 crore; Prof

In [18]:
context=vectorstore.similarity_search('What test_123 company and location?',k=3)


In [19]:
response=llm.invoke(f'test_123 company location?: {context}')
print(response.content)

Location: Faridabad, Haryana, India.

The document states the company is "Established in the bustling industrial hub of Faridabad, Haryana." If you need an exact address, it isn’t provided in the snippet.
